# Predict

> Fill in a module description here

In [ ]:
#| default_exp prediction.predict

In [ ]:
#hide

%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

%matplotlib inline

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pympp.process import *

In [ ]:
#| export
from pathlib import Path

In [ ]:
bpi12 ='../data/logs/csv/mppn_ds/BPIC12.csv'

In [ ]:
log=import_log(bpi12)
log

,event_id,resource,timestamp,activity,REG_DATE,AMOUNT_REQ
trace_id,,,,,,
173688,0,112.0,2011-09-30 22:38:44.546000+00:00,###start###,2011-10-01 00:38:44.546000+02:00,20000
173688,1,112.0,2011-09-30 22:38:44.546000+00:00,A_SUBMITTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,2,112.0,2011-09-30 22:38:44.880000+00:00,A_PARTLYSUBMITTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,3,112.0,2011-09-30 22:39:37.906000+00:00,A_PREACCEPTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,4,112.0,2011-09-30 22:39:38.875000+00:00,W_Completeren aanvraag_SCHEDULE,2011-10-01 00:38:44.546000+02:00,20000
...,...,...,...,...,...,...
214376,3,112.0,2012-02-29 22:52:01.287000+00:00,W_Afhandelen leads_SCHEDULE,2012-02-29 23:51:16.799000+01:00,15000
214376,4,11169.0,2012-03-01 08:26:46.736000+00:00,W_Afhandelen leads_START,2012-02-29 23:51:16.799000+01:00,15000
214376,5,11169.0,2012-03-01 08:27:37.118000+00:00,A_DECLINED_COMPLETE,2012-02-29 23:51:16.799000+01:00,15000


## Sample Training

In [ ]:
col='activity'
o=PPObj(log,procs=Categorify(),cat_names=col,y_names=col,splits=split_traces(log))
o.show()

#traces: 12616 #events: 249149


,activity
trace_id,
193068,2
193068,12
193068,9


In [ ]:
dls=o.get_dls()
xb,yb=dls.one_batch()
xb.shape,yb[0].shape

(torch.Size([64, 1, 5]), torch.Size([64]))

In [ ]:
path = "../data/logs/csv/mppn_ds/Helpdesk.csv"

In [ ]:
log=import_log(path)
log.head()

,event_id,timestamp,activity,resource
trace_id,,,,
Case1,0,2012-10-09 11:50:17+00:00,###start###,Value 1
Case1,1,2012-10-09 11:50:17+00:00,Assign seriousness,Value 1
Case1,2,2012-10-09 11:51:01+00:00,Take in charge ticket,Value 1
Case1,3,2012-10-12 12:02:56+00:00,Take in charge ticket,Value 2
Case1,4,2012-10-25 08:54:26+00:00,Resolve ticket,Value 1


In [ ]:
col='activity'
o=PPObj(log,procs=Categorify(),cat_names=col,y_names=col,splits=split_traces(log))
o.show()

#traces: 4580 #events: 30508


,activity
trace_id,
Case3756,2
Case3756,3
Case3756,14


In [ ]:
dls=o.get_dls()
xb,yb=dls.one_batch()
xb.shape,yb[0].shape,xb[0], len(o.procs.categorify[col])

(torch.Size([64, 1, 5]),
 torch.Size([64]),
 tensor([[ 0,  2,  3, 14, 16]], device='cuda:0'),
 17)

In [ ]:
#| export
from math import sqrt
from fastai.torch_basics import *
from fastai.metrics import accuracy
from fastai.callback.all import *
from fastai.learner import *


In [ ]:
#| export
class RNNwEmbedding(torch.nn.Module) :
    def __init__(self,o) :
        super().__init__()
        vocab_size=len(o.procs.categorify[o.y_names[0]])
        emb_size = int(sqrt(vocab_size))+1
        hidden_six = 20
        self.emb = nn.Embedding(vocab_size,emb_size)

        self.rnn = nn.RNN(emb_size, hidden_six, batch_first=True, num_layers=2)

        self.linear = nn.Linear(hidden_six, vocab_size)

    def forward(self, x):
        x = self.emb(x.squeeze())
        x,_ = self.rnn(x)
        x = x[:,-1]
        x = self.linear(x)
        x = F.softmax(x,dim=1)
        return x

In [ ]:
m=RNNwEmbedding(o)


In [ ]:
#| export
def cross_entropy_targ0(inp, targ):
    return F.cross_entropy(inp,targ[0])

def accuracy_targ0(inp, targ, axis=-1):
    "Compute accuracy with `targ` when `pred` is bs * n_classes"
    pred,targ = flatten_check(inp.argmax(dim=axis), targ[0])
    return (pred == targ).float().mean()

In [ ]:
#| export
class HideOutput:
    'A utility function that hides all outputs in a context'
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [ ]:
p=m(xb.cpu())

In [ ]:
#| export
def training_loop(learn,epoch,print_output,lr_find):
    '''
    Basic training loop that uses learning rate finder and one cycle training. 
    See fastai docs for more information
    '''
    if lr_find:
        lr=np.median([learn.lr_find(show_plot=print_output)[0] for i in range(5)])
        learn.fit_one_cycle(epoch,float(lr))
    else: learn.fit(epoch,0.01)

In [ ]:
#| export
def train_validate(dls,m,metrics=accuracy,loss=F.cross_entropy,epoch=20,print_output=True,model_dir=".",lr_find=True,
                   output_index=1,patience=3,min_delta=0.005,show_plot=True,store_path='tmp',model_name='.model'):
    '''
    Trains a model on the training set with early stopping based on the validation loss.
    Afterwards, applies it to the test set.
    '''
    cbs = [
      EarlyStoppingCallback(monitor='valid_loss',min_delta=min_delta, patience=patience),
      SaveModelCallback(fname=model_name)
      ]
    learn=Learner(dls, m, path=store_path, model_dir=model_dir, loss_func=loss ,metrics=metrics,cbs=cbs)

    if print_output:
        training_loop(learn,epoch,show_plot,lr_find=lr_find)
        return learn.validate(dl=dls[2])[output_index]
    else:
        with HideOutput(),learn.no_bar(),learn.no_logging():
            training_loop(learn,epoch,show_plot,lr_find=lr_find)
            return learn.validate(dl=dls[2])[output_index]

In [ ]:
cols,outcome='activity',False
o=PPObj(log,procs=Categorify(),cat_names=cols,y_names=cols,splits=split_traces(log))
dls=o.get_dls(outcome=outcome,windows=partial(windows_fast))
m=RNNwEmbedding(o)
train_validate(dls,m,epoch=5,metrics=accuracy_targ0,loss=lambda x,y: F.cross_entropy(x,y[0]),print_output=False,show_plot=False)

0.858601450920105

## Process Prediction Model


The `PPModel` class creates multiple prediction models for next-step prediction, next-resource prediction, remaining time prediction, etc. based on a pytorch model architecture.

In [ ]:
#| export

# Todo: Add Logging
class PPModel():

    def __init__(self,log,ds_name,splits,store=None,bs=64,print_output=False,patience=3,min_delta=0.005,
                 attr_dict=None,windows=windows_fast,epoch=20,sample=False,
                 train_validate=train_validate):
        store_attr('log,ds_name,splits,attr_dict,windows,epoch,bs,print_output,min_delta,patience,store')
        self.lr_find=True
        if sample:
            self.lr_find=False
            traces=self.splits[0]
            self.splits=traces[:60],traces[60:80],traces[80:100]
            self.bs=64
            self.epoch=1

    def evaluate(self):
        if not self.print_output:
            with HideOutput(): return self.__evaluate()
        else: return self.__evaluate()
    def __evaluate(self):
        print(self.ds_name,self.get_name())
        self.setup()

        print('next_step_prediction')
        nsp=self.next_step_prediction()

        print('next_resource_prediction')
        nrp=self.next_resource_prediction()

        print('last_resource_prediction')
        lrp=self.last_resource_prediction()

        print('outcome_prediction')
        op=self.outcome_prediction()

        print('duration_to_next_event_prediction')
        dtnep=self.duration_to_next_event_prediction()

        print('duration_to_end_prediction')
        dtep=self.duration_to_end_prediction()

        print('activity_suffix_prediction')
        asp=self.activity_suffix_prediction()

        print('resource_suffix_prediction')
        rsp=self.resource_suffix_prediction()

        return nsp, nrp, lrp, op, dtnep, dtep, asp, rsp

    def _train_validate(self,dls,m,metrics=accuracy,loss=F.cross_entropy,output_index=1):
        store,model_name='tmp','.model'
        if self.store:
            ins_stack=inspect.stack()
            model_name=str(ins_stack[2][3]) if str(ins_stack[2][3])!='__evaluate' else str(ins_stack[1][3])
            store=self.store/self.ds_name/self.get_name()
        return train_validate(dls,m,metrics=metrics,loss=loss,output_index=output_index, #Only change these
                              epoch=self.epoch,print_output=self.print_output,patience=self.patience,
                              min_delta=self.min_delta,show_plot=False,store_path=store,model_name=model_name,
                              lr_find=self.lr_find)
    def setup(self): pass
    def get_name(self): return self.__class__.__name__.replace('PPM_',"")
    def next_step_prediction(self): pass
    def next_resource_prediction(self): pass
    def last_resource_prediction(self): pass
    def outcome_prediction(self): pass
    def duration_to_next_event_prediction(self): pass
    def duration_to_end_prediction(self): pass
    def activity_suffix_prediction(self): pass
    def resource_suffix_prediction(self): pass

In [ ]:
#| export
def get_ds_name(url): return(Path(url).stem) # Utility function, that gets the name of a dataset

In [ ]:
#| export
class PPM_RNNwEmbedding(PPModel):
    'Sampe PPM based on RNNwEmbedding'
    model=RNNwEmbedding

    def next_step_prediction(self,outcome=False,col='activity'):
        o=PPObj(self.log,procs=Categorify(),cat_names=col,y_names=col,splits=self.splits)
        dls=o.get_dls(outcome=outcome,bs=self.bs,windows=self.windows)
        m=self.model(o)
        return self._train_validate(dls,m,metrics=accuracy_targ0,loss=cross_entropy_targ0)

    def next_resource_prediction(self): return self.next_step_prediction(col='resource')
    def last_resource_prediction(self): return self.next_step_prediction(col='resource',outcome=True)
    def outcome_prediction(self): return self.next_step_prediction(outcome=True)

In [ ]:
log=import_log(path)
ds_name=get_ds_name(path)
splits=split_traces(log)

In [ ]:
ppm=PPM_RNNwEmbedding(log,ds_name,splits,epoch=1)

In [ ]:
ppm.next_step_prediction()

0.853634774684906

## Pipeline Runner

A runner function for `PPModel`. Runs multiple prediction models on various datasets for several runs. Writes the results to a dataframe and stores it on disk in the `./tmp` folder. Stores the splits, the trained models, and the results. Accepts multiple parameters.

In [ ]:
#| export
from datetime import datetime
import inspect
from tqdm.notebook import tqdm

In [ ]:
#| export
class Performance_Statistic():
    'Creates a results dataframe, that shows the performance of all models on all datasets on all tasks.'
    def __init__(self):
        self.df = pd.DataFrame(
        columns=['Dataset', 'Model', 'Next Step', 'Next Resource', 'Last Resource', 'Outcome',
                'Next relative Timestamp', 'Duration to Outcome', 'Activity Suffix', 'Resource Suffix'])
    def update(self,model_performance): self.df.loc[len(self.df)] = model_performance
    def to_df(self):
        return self.df

In [ ]:
#| export
def _store_path(results_dir=Path('./tmp')):
    'Creates a new folder to store results'
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d_%H:%M:%S")
    results_dir=results_dir/current_time
    results_dir.mkdir()
    return results_dir

In [ ]:
#| export
@delegates(PPModel)
def runner(dataset_urls,ppm_classes,store=True,runs=1,sample=False,seed=42,tqdm=tqdm,
           **kwargs):
    store_path= _store_path() if store else None
    '''
    Runs a number of process prediction models PPModel on a number of datasets for multiple runs.
    Stores results in ./tmp folder.
    '''
    i=0
    results=[]
    for r in tqdm(range(runs),desc='Runs'):
        performance_statistic = Performance_Statistic()
        db=tqdm(range(len(dataset_urls)),leave=False)
        for i in db:
            db.set_description(get_ds_name(dataset_urls[i]))
            ds= dataset_urls[i]
            log=import_log(ds)
            ds_name=get_ds_name(ds)
            splits=split_traces(log,ds_name,seed=seed)
            if store:
                with open(store_path/f'run{r}_{ds_name}_splits.pickle', "wb") as output_file:
                    pickle.dump(splits, output_file)
            mb=tqdm(range(len(ppm_classes)),leave=False)
            for j in mb:
                mb.set_description(ppm_classes[j].__name__.replace('PPM_',""))
                ppm_class=ppm_classes[j]
                model_path=store_path/'models'/f"run{r}" if store else None
                model=ppm_class(log,ds_name,splits,store=model_path,sample=sample,**kwargs)
                model_performance = model.evaluate()
                model_performance = [ds_name, model.get_name(),*model_performance]
                performance_statistic.update(model_performance)
                [ds_name, model.get_name(),*model_performance]

        df = performance_statistic.to_df()
        results.append(df)
        if store: df.to_csv(store_path/f"run_{r}_results.csv")
    return results if len(results)>1 else results[0]

This sample shows how to use the runner function:

In [ ]:
%%time
dataset_paths=[path]
ppms=[PPM_RNNwEmbedding,PPM_RNNwEmbedding]
res=runner(dataset_paths,ppms,epoch=1,store=True,print_output=False,
           sample=True,runs=2)


Runs:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 2.92 s, sys: 23.1 s, total: 26 s
Wall time: 27.7 s


In [ ]:
print ("RUN",0)
display_df(res[0])

print ("RUN",1)
display_df(res[1])

RUN 0


,Dataset,Model,Next Step,Next Resource,Last Resource,Outcome,Next relative Timestamp,Duration to Outcome,Activity Suffix,Resource Suffix
0,Helpdesk,RNNwEmbedding,0.525424,0.322034,0.694915,1.0,None,None,None,None
1,Helpdesk,RNNwEmbedding,0.635593,0.254237,0.694915,1.0,None,None,None,None


RUN 1


,Dataset,Model,Next Step,Next Resource,Last Resource,Outcome,Next relative Timestamp,Duration to Outcome,Activity Suffix,Resource Suffix
0,Helpdesk,RNNwEmbedding,0.313559,0.262712,0.694915,1.0,None,None,None,None
1,Helpdesk,RNNwEmbedding,0.177966,0.245763,0.694915,1.0,None,None,None,None


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
import torch



In [ ]:
torch.cuda.is_available()


True

In [ ]:
torch.cuda.device_count()



1